In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import optuna

In [3]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import files
 
 
uploaded = files.upload()

Saving 3-Admission_Prediction.csv to 3-Admission_Prediction (2).csv


In [6]:
df = pd.read_csv('3-Admission_Prediction.csv')
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [7]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [8]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [9]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [10]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [11]:
x = df.drop(columns = ['Serial No.','Chance of Admit'])
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.000000,118.0,4.0,4.5,4.5,9.65,1
1,324.000000,107.0,4.0,4.0,4.5,8.87,1
2,316.558763,104.0,3.0,3.0,3.5,8.00,1
3,322.000000,110.0,3.0,3.5,2.5,8.67,1
4,314.000000,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.000000,108.0,5.0,4.5,4.0,9.02,1
496,337.000000,117.0,5.0,5.0,5.0,9.87,1
497,330.000000,120.0,5.0,4.5,5.0,9.56,1
498,312.000000,103.0,4.0,4.0,5.0,8.43,0


In [12]:
y = df['Chance of Admit']
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [13]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x = std_sca.fit_transform(x)

In [16]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [14]:
from sklearn.metrics import mean_squared_error
#objective function
def objective(trail , data = x , target = y):
  x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = .10, random_state = 30)
  param = {
      'tree_method' : 'gpu_hist',
      'lambda' : trail.suggest_loguniform('lambda', 1e-4,10.0),
      'alpha' : trail.suggest_loguniform('alpha', 1e-4,10.0),
      'colsample_bytree' : trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'subsample' : trail.suggest_categorical('subsample', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
      'learning_rate' : trail.suggest_categorical('learning_rate', [.00001, .0004, .009, .02, .1, 1, 5, 10]),
      'n_estimator' : 30000,
      'max_depth' : trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
      'random_state' : trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454, 243123]),
      'min_child_weight' : trail.suggest_int('min_child_weight' ,1,200),
  }
  xgb_reg_model = xgb.XGBRegressor(**param)
  xgb_reg_model.fit(x_train,y_train,eval_set = [(x_test,y_test)] , verbose = True )
  pred_xgb = xgb_reg_model.predict(x_test)
  rmse = mean_squared_error(y_test,pred_xgb)
  return rmse

In [21]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [15]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective, n_trials = 10)
find_param.best_trial.params

[I 2022-07-23 07:59:58,392] A new study created in memory with name: no-name-12469215-5b47-4d8b-9803-7259d4f7f54f


[07:59:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.8856
[1]	validation_0-rmse:16.2211
[2]	validation_0-rmse:133.628
[3]	validation_0-rmse:1134.57
[4]	validation_0-rmse:10410.2
[5]	validation_0-rmse:92996.9
[6]	validation_0-rmse:835348
[7]	validation_0-rmse:7.49917e+06
[8]	validation_0-rmse:6.73267e+07
[9]	validation_0-rmse:6.04447e+08
[10]	validation_0-rmse:5.42662e+09
[11]	validation_0-rmse:4.87193e+10
[12]	validation_0-rmse:4.37393e+11
[13]	validation_0-rmse:3.92684e+12
[14]	validation_0-rmse:3.52545e+13
[15]	validation_0-rmse:3.16509e+14
[16]	validation_0-rmse:2.84156e+15
[17]	validation_0-rmse:2.5511e+16
[18]	validation_0-rmse:2.29034e+17
[19]	validation_0-rmse:2.05622e+18
[20]	validation_0-rmse:inf
[21]	validation_0-rmse:inf
[22]	validation_0-rmse:inf
[23]	validation_0-rmse:inf
[24]	validation_0-rmse:inf
[25]	validation_0-rmse:inf
[26]	validation_0-rmse:inf
[27]	validation_0-rmse:in

[W 2022-07-23 07:59:59,227] Trial 0 failed because of the following error: ValueError("Input contains NaN, infinity or a value too large for dtype('float32').")
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-14-a6fc4219c642>", line 20, in objective
    rmse = mean_squared_error(y_test,pred_xgb)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 439, in mean_squared_error
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py", line 800, in check_array
    _assert_all_finite(array, allow_nan=force_all_finite == "allow-nan")
  File "/usr/local/lib/python3.7/dist-packages/sklearn/utils/valida

ValueError: ignored

In [30]:
#running above code will give best parameter of lambda, alpha, .......

In [ ]:
xgb.XGBRFRegressor()